In [10]:
script_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/SCRIPT/000091.TXT'
new_script_dir = '/home/dgxuser/Desktop/kailin/script_clean.txt'

with open(new_script_dir, 'w') as fout: 
    with open(script_dir, encoding = 'utf-8') as script:
        script.readline()
        line = script.readline().strip()
        while True:
            line = script.readline()
            if line == '': 
                break
            line = script.readline().strip()
            words = line.split()
            edited_words = []
            for word in words:
                if (word[-1] != '>') and (word != '**'):
                    edited_words.append(word.lower())
            transcript_line = ""
            for word in edited_words: 
                transcript_line = transcript_line + word + " "
            transcript_line = transcript_line[:-1]
            fout.write(transcript_line)
            fout.write('\n')

In [12]:
## Importing libraries 
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
import numpy as np
import soundfile as sf
from pathlib import Path
from nemo.collections.tts.torch.helpers import BetaBinomialInterpolator
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from pydub import AudioSegment

from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.models import FastPitchModel

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

[NeMo I 2022-07-12 15:30:11 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-12 15:30:11 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-12 15:30:11 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-12 15:30:14 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-12 15:30:14 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-12 15:30:14 features:200] PADDING: 0


[NeMo W 2022-07-12 15:30:14 features:177] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-12 15:30:14 features:200] PADDING: 0
[NeMo I 2022-07-12 15:30:15 save_restore_connector:243] Model HifiGanModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [13]:
def infer(spec_gen_model, vocoder_model, str_input):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=None)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def get_best_ckpt_from_last_run(
        base_dir, 
        new_speaker_id, 
        mixing_enabled, 
        model_name="FastPitch"
    ):    
    mixing = "no_mixing" if not mixing_enabled else "mixing"
    
    d = f"S{new_speaker_id}_{mixing}"
    
    exp_dirs = list([i for i in (Path(base_dir) / d / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

In [16]:
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/Part2/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifest2.json'

n = 0
speakers = [2001, 2002, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011]

with open(manifest_path, 'w') as fout: 
    for speaker_id in speakers: 
        mixing = False
        last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
        print(last_ckpt)
        spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
        spec_model.eval().cuda()

        with open(text_dir, "r") as f: 
            while True: 
              text = f.readline().strip()
              if text == '': 
                break 
              spec, audio = infer(spec_model, vocoder, text)
              audio = ipd.Audio(audio, rate=22050)
              audio = AudioSegment(audio.data)
              file_dir = audio_dir + str(n) + '.wav'
              audio.export(file_dir, format="wav")

              duration = librosa.core.get_duration(filename=file_dir)
              metadata = {
                "audio_filepath": file_dir,
                "duration": duration, 
                "text": text
              }
              json.dump(metadata, fout)
              fout.write('\n')   
              n = n + 1 

S2001_no_mixing/FastPitch/2022-07-04_11-53-09/checkpoints/FastPitch--v_loss=1.1350-epoch=24-last.ckpt
[NeMo I 2022-07-12 15:52:00 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-12 15:52:35 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-12 15:52:35 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL1/manifest_S2001_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min

[NeMo I 2022-07-12 15:52:35 features:200] PADDING: 1
S2002_no_mixing/FastPitch/2022-07-07_10-47-10/checkpoints/FastPitch--v_loss=1.3819-epoch=24-last.ckpt
[NeMo I 2022-07-12 15:53:18 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-12 15:53:53 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-12 15:53:53 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL1/melSpect_S2002_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min

[NeMo I 2022-07-12 15:53:53 features:200] PADDING: 1
S2003_no_mixing/FastPitch/2022-07-04_12-08-34/checkpoints/FastPitch--v_loss=0.8842-epoch=49-last.ckpt
[NeMo I 2022-07-12 15:54:37 tokenize_and_classify:87] Creating ClassifyFst grammars.


...

[NeMo I 2022-07-12 16:02:02 features:200] PADDING: 1
S2010_no_mixing/FastPitch/2022-07-04_14-58-14/checkpoints/FastPitch--v_loss=1.4038-epoch=24-last.ckpt
[NeMo I 2022-07-12 16:02:50 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-12 16:03:26 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-12 16:03:26 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL1/manifest_S2010_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min

[NeMo I 2022-07-12 16:03:26 features:200] PADDING: 1
S2011_no_mixing/FastPitch/2022-07-04_15-13-24/checkpoints/FastPitch--v_loss=1.1233-epoch=49-last.ckpt
[NeMo I 2022-07-12 16:04:07 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-12 16:04:42 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-12 16:04:42 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL1/manifest_S2011_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min

[NeMo I 2022-07-12 16:04:42 features:200] PADDING: 1


In [17]:
## Importing libraries 
import nemo
import nemo.collections.asr as nemo_asr
import jiwer
from jiwer import wer
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
from omegaconf import OmegaConf, open_dict
from nemo.collections.asr.models.ctc_bpe_models import EncDecCTCModelBPE
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager

In [18]:
# Initialise quartznet model 
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5")

[NeMo I 2022-07-12 16:05:51 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2022-07-12 16:05:51 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2022-07-12 16:05:51 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-12 16:05:52 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-07-12 16:05:52 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2022-07-12 16:05:52 features:200] PADDING: 16
[NeMo I 2022-07-12 16:05:53 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.


In [19]:
import json 
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifest2.json'
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
error = wer(transcript, predicted)
print('Word Error Rate: ' + str(error))

Word Error Rate: 0.23253871585174873


In [21]:
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/Part1/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifest1.json'

n = 0
speakers = [91, 124, 135, 148, 173, 178, 194, 208, 289, 245]

with open(manifest_path, 'w') as fout: 
    for speaker_id in speakers: 
        mixing = False
        last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
        print(last_ckpt)
        spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
        spec_model.eval().cuda()

        with open(text_dir, "r") as f: 
            while True: 
              text = f.readline().strip()
              if text == '': 
                break 
              spec, audio = infer(spec_model, vocoder, text)
              audio = ipd.Audio(audio, rate=22050)
              audio = AudioSegment(audio.data)
              file_dir = audio_dir + str(n) + '.wav'
              audio.export(file_dir, format="wav")

              duration = librosa.core.get_duration(filename=file_dir)
              metadata = {
                "audio_filepath": file_dir,
                "duration": duration, 
                "text": text
              }
              json.dump(metadata, fout)
              fout.write('\n')   
              n = n + 1 

S91_no_mixing/FastPitch/2022-07-07_16-05-57/checkpoints/FastPitch--v_loss=1.1729-epoch=24-last.ckpt
[NeMo I 2022-07-12 16:26:37 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-12 16:27:10 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-12 16:27:10 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/melSpect_S91_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_d

[NeMo I 2022-07-12 16:27:10 features:200] PADDING: 1
S124_no_mixing/FastPitch/2022-07-07_16-30-35/checkpoints/FastPitch--v_loss=2.6331-epoch=49-last.ckpt
[NeMo I 2022-07-12 16:28:06 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-12 16:28:42 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-12 16:28:42 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S124_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_

[NeMo I 2022-07-12 16:28:43 features:200] PADDING: 1
S135_no_mixing/FastPitch/2022-07-07_16-50-37/checkpoints/FastPitch--v_loss=2.2859-epoch=49-last.ckpt
[NeMo I 2022-07-12 16:29:49 tokenize_and_classify:87] Creating ClassifyFst grammars.


...

[NeMo I 2022-07-12 16:38:36 features:200] PADDING: 1
S289_no_mixing/FastPitch/2022-07-12_09-59-53/checkpoints/FastPitch--v_loss=0.7410-epoch=24-last.ckpt
[NeMo I 2022-07-12 16:39:18 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-12 16:39:53 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-12 16:39:53 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S289_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_

[NeMo I 2022-07-12 16:39:53 features:200] PADDING: 1
S245_no_mixing/FastPitch/2022-07-12_10-14-25/checkpoints/FastPitch--v_loss=0.9827-epoch=49-last.ckpt
[NeMo I 2022-07-12 16:40:35 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-12 16:41:13 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-12 16:41:13 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S245_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_

[NeMo I 2022-07-12 16:41:13 features:200] PADDING: 1


In [22]:
import json 
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifest1.json'
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
error = wer(transcript, predicted)
print('Word Error Rate: ' + str(error))

Word Error Rate: 0.28862014964329213
